In [1]:
import folium
import pandas as pd
import json
import os

### json 파일 출처
* [Github : achimkoh](https://github.com/southkorea/southkorea-maps/blob/master/kostat/2018/json/skorea-provinces-2018-topo.json?short_path=f0e2487)

In [4]:
class High_school_map:
    def __init__(self, office_of_education_csv_name):
        self.csv_name = office_of_education_csv_name
        self.csv_path = '../data_csv/' + self.csv_name

        self.high_school_dataframe = pd.read_csv(self.csv_path)
        
        # Unnamed: 0 삭제하고 Unnamed: 0 열을 인덱스로 함
        save_index = list(self.high_school_dataframe['Unnamed: 0'])
        self.high_school_dataframe.drop(['Unnamed: 0'], axis=1, inplace=True)
        self.high_school_dataframe.index = save_index

        self.save_file_name = '' # 저장할 때 파일 이름

        self.set_map()
        
    # 지도 준비 or 초기화
    def set_map(self):
        self.save_file_name = 'init_map'

        # 위도, 경도의 평균을 지도의 중심으로 지정
        sum_latitude = float()
        sum_longitude = float()

        df = self.high_school_dataframe

        for i in df.index:
            sum_latitude += float(df.loc[i, '위도'])
            sum_longitude += float(df.loc[i, '경도'])
        
        mean_latitude = sum_latitude / float(len(df.loc[:, '위도']))
        mean_longitude = sum_longitude / float(len(df.loc[:, '경도']))

        with open('../code/행정구역.json', mode= 'rt', encoding='utf-8') as file:
            geo = json.loads(file.read())
            file.close()

        # 지도 설정
        self.high_school_map = folium.Map(location=[mean_latitude, mean_longitude],
        zoom_start=9)

        folium.GeoJson(
            geo,
            name='korea'
        ).add_to(self.high_school_map)

    def __get_popup_html(self, school_name, coordinate, address,
                        phone, total_student, boy, girl, 
                        homepage, school_type, affiliations):

        show_student_data = '전체 : ' + total_student
        if boy == '0명' and girl == '0명':
            pass
        else:
            show_student_data += ' / 남 : ' + boy + ' & 여 : ' + girl
        
        show_school_type = '<span>' + school_type + '</span>'
        show_affiliations = '<span>' + '/ 계열 : ' + affiliations + '</span>'

        html_popup = """
            <!DOCTYPE html>
            <html>
                <title>
                    
                </title>
                <body>
                    <h3 style="display:inline">""" + school_name + """</h3>
                    """ + show_school_type + show_affiliations + """
                    <br></br>
                    <table style="height: 126px; width: 500px;">

                        <tr>
                            <th>좌표</th>
                            <td>위도 : """ + str(coordinate[0]) + '° / ' + '경도 : ' + str(coordinate[1]) + '°' + """</td>
                        </tr>
                        <tr>
                            <th>주소</th>
                            <td>""" + address + """</td>
                        </tr>
                        <tr>
                            <th>전화번호</th>
                            <td>""" + phone + """</td>
                        </tr>
                        <tr>
                            <th>재학생 수</th>
                            <td>""" + show_student_data + """</td>
                        </tr>
                        <tr>
                            <th>홈페이지</th>
                            <td><a href=""" + '"' + homepage + '"' + """>""" + homepage + """</a></td>
                        </tr>
                    </table>
                <body>
            </html> 
        """

        return html_popup

    def sample(self):
        # 그냥 표시
        self.save_file_name = 'sample'

        df = self.high_school_dataframe

        for i in df.index:
            coordinate = df.loc[i,'위도'], df.loc[i, '경도']
            marking = folium.Marker(
                location= coordinate,
                tooltip= i
            ).add_to(self.high_school_map)

    def according_to_school_type(self):
        df = self.high_school_dataframe

        self.save_file_name = 'according_to_school_type'

        for i in df.index:
            if df.loc[i, '여학생 수'] == '0명': # 남고 식별
                color = 'blue'
                school_type = '남고'

            elif df.loc[i, '남학생 수'] == '0명': # 여고 식별
                color = 'red'
                school_type = '여고'

            else: # 둘 다 아니면 남녀 공학
                color = 'gray'
                school_type = '남녀 공학'

            coordinate = df.loc[i, '위도'], df.loc[i, '경도']

            html = self.__get_popup_html(i, coordinate, df.loc[i, '주소'],
                                        df.loc[i, '전화번호'], df.loc[i, '재학생 수'],
                                        df.loc[i, '남학생 수'], df.loc[i, '여학생 수'],
                                        df.loc[i, '홈페이지'], school_type, df.loc[i, '계열'])

            popup = folium.Popup(html, min_wdith=200, max_width=1000)

            icon = folium.Icon(color= color)

            marking = folium.Marker(
                location= coordinate,
                popup= popup,
                tooltip= i,
                icon= icon
            ).add_to(self.high_school_map)

    def save_to_html(self):
        html_file_name = self.csv_name.strip('.csv') + ' 지도' + '(' + self.save_file_name + ').html'
        path = '../maps/'

        self.high_school_map.save(path + html_file_name)

In [6]:
csv_file_path = '../data_csv/'

file_list = os.listdir(csv_file_path)

for i in file_list:
    high_school_map = High_school_map(i)

    high_school_map.according_to_school_type()

    high_school_map.save_to_html()

    del high_school_map